# Exploration of Data.gov.sg's weather and rainfall API

Ref: https://beta.data.gov.sg/collections/1459/view

Data of interest: Realtime air temperature and rainfall.
From a sample response of realtime air temperature, it was noted that there are only 3 sensors that detects the air temperature, whereas for rainfall, there are 64 of such.

In [47]:
# Library imports
from random import randint
from typing import Union, Dict
import requests
import numpy as np

### Helper functions

In [51]:
def api_query(api_link: str,  agent_id: str) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header

    Returns:
        Dictioanry containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id}
    try:
        res = requests.get(api_link,
                            headers=req_headers,
                            timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print(f"Request sucessful with {res.status_code}")
            the_json = res.json()
        return the_json

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

def query_average_air_temperature(api_link: str = "https://api.data.gov.sg/v1/environment/air-temperature", agent_id: str= "test_qzq"): 
    air_temp_response = api_query(api_link=api_link, agent_id=agent_id)
    if not air_temp_response:
        print("No response received")
        return {}
    else:
        station_temp_readings_list = air_temp_response["items"][0]["readings"]
        print(f"Number of air temperature sensor stations: {len(station_temp_readings_list)}")
        actual_readings = [readings_dict["value"] for readings_dict in station_temp_readings_list]
        
        min_reading, max_reading = np.min(actual_readings), np.max(actual_readings)
    return min_reading, max_reading



def query_average_rainfall(api_link: str = "https://api.data.gov.sg/v1/environment/rainfall", agent_id: str= "test_qzq"): 
    new_rainfall_dict = {}
    rainfall_response = api_query(api_link=api_link, agent_id=agent_id)
    if not rainfall_response:
        print("No response received")
        return new_rainfall_dict
    else:
        stations_list = rainfall_response["metadata"]["stations"]
        print(f"Number of rainfall stations: {len(stations_list)}")
        readings = rainfall_response["items"][0]["readings"]
        for station_dict, reading_dict in zip(stations_list, readings):
            # Pop name key from statiio
            name = station_dict.pop("name")
            location = station_dict.pop("location")
            lat, lon = location["latitude"], location["longitude"]
            value = reading_dict["value"]
            if name not in new_rainfall_dict:
                new_rainfall_dict[name] = station_dict
                new_rainfall_dict[name]["latitude"] = lat
                new_rainfall_dict[name]["longitude"] = lon
                new_rainfall_dict[name]["value"] = value
            # Update the value
            else:
                new_rainfall_dict[name]["value"] = value            
    return new_rainfall_dict

### Function call to explore response outputs

In [52]:
query_average_air_temperature()

Request sucessful with 200
Number of air temperature sensor stations: 3


(25.6, 27.0)

In [53]:
query_average_rainfall()

Request sucessful with 200
Number of rainfall stations: 64


{'Alexandra Road': {'id': 'S77',
  'device_id': 'S77',
  'latitude': 1.2937,
  'longitude': 103.8125,
  'value': 0},
 'Ang Mo Kio Avenue 5': {'id': 'S109',
  'device_id': 'S109',
  'latitude': 1.3764,
  'longitude': 103.8492,
  'value': 0},
 'Banyan Road': {'id': 'S117',
  'device_id': 'S117',
  'latitude': 1.256,
  'longitude': 103.679,
  'value': 0},
 'Bukit Panjang Road': {'id': 'S64',
  'device_id': 'S64',
  'latitude': 1.3824,
  'longitude': 103.7603,
  'value': 0},
 'Bukit Timah Road': {'id': 'S90',
  'device_id': 'S90',
  'latitude': 1.3191,
  'longitude': 103.8191,
  'value': 0},
 'Choa Chu Kang Avenue 4': {'id': 'S114',
  'device_id': 'S114',
  'latitude': 1.38,
  'longitude': 103.73,
  'value': 0},
 'Clementi Road': {'id': 'S50',
  'device_id': 'S50',
  'latitude': 1.3337,
  'longitude': 103.7768,
  'value': 0},
 'East Coast Parkway': {'id': 'S107',
  'device_id': 'S107',
  'latitude': 1.3135,
  'longitude': 103.9625,
  'value': 0},
 'GEYLANG EAST CENTRAL': {'id': 'S215',
  '